## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
from pyspark.sql.types import *

from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

from pyspark.sql.functions import *
from pyspark.ml.feature import *
from pyspark.ml import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.mllib.evaluation import *

In [0]:
IS_DB = True # Run the code in Databricks

PYSPARK_CLI = False
if PYSPARK_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

In [0]:
EcommerceSchema = StructType([
  StructField("event_time", TimestampType(), False),
  StructField("event_type", StringType(), False),
  StructField("product_id", IntegerType(), False),
  StructField("category_id", LongType(), False),
  StructField("StringType", StringType(), False),
  StructField("brand", StringType(), False),
  StructField("price", DoubleType(), False),
  StructField("user_id", IntegerType(), False),
  StructField("user_session", StringType(), False),
])

### Load Source Data
The data for this exercise is provided as a CSV file containing details of e-commerce items and catetegories. The data includes specific characteristics (or *features*) for each item, as well as a *label* column indicating what is the Event Type of each item.

You will load this data into a DataFrame and display it.

In [0]:
#For hadoop File location
#file_location = "/user/apandey9/5560/2019-Oct_Master.csv"
#file_type = "csv"

#This is sample File Location
# File location and type
file_location = "/FileStore/tables/2019_Oct-1.txt"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location) 

df = df.withColumnRenamed('# File format is event_time', 'event_time')
df = df.filter((col("brand") != "No value") & (col("category_code") != "No value"))


In [0]:
df = df.withColumn("event_type", when(df.event_type == 'view', 1) \
                  .when(df.event_type == 'cart', 2) \
                  .when(df.event_type == 'purchase', 3))
df.display()

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01T00:00:00.000+0000,1,3900821,2053013552326770000,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2019-10-01T00:00:01.000+0000,1,1307067,2053013558920210000,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
2019-10-01T00:00:04.000+0000,1,1004237,2053013555631880000,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
2019-10-01T00:00:05.000+0000,1,1480613,2053013561092860000,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
2019-10-01T00:00:10.000+0000,1,28719074,2053013565480100000,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
2019-10-01T00:00:11.000+0000,1,1004545,2053013555631880000,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb
2019-10-01T00:00:11.000+0000,1,2900536,2053013554776240000,appliances.kitchen.microwave,elenberg,51.46,555158050,b5bdd0b3-4ca2-4c55-939e-9ce44bb50abd
2019-10-01T00:00:11.000+0000,1,1005011,2053013555631880000,electronics.smartphone,samsung,900.64,530282093,50a293fb-5940-41b2-baf3-17af0e812101
2019-10-01T00:00:13.000+0000,1,3900746,2053013552326770000,appliances.environment.water_heater,haier,102.38,555444559,98b88fa0-d8fa-4b9d-8a71-3dd403afab85
2019-10-01T00:00:16.000+0000,1,13500240,2053013557099880000,furniture.bedroom.bed,brw,93.18,555446365,7f0062d8-ead0-4e0a-96f6-43a0b79a2fc4


In [0]:
#Converting String Values into Index Values
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="brand", outputCol="brandIndex")
indexer1 = StringIndexer(inputCol="category_code", outputCol="category_codeIndex")
df = indexer.fit(df).transform(df) 
df = indexer1.fit(df).transform(df)
df.display()

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,brandIndex,category_codeIndex
2019-10-01T00:00:00.000+0000,1,3900821,2053013552326770000,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,144.0,26.0
2019-10-01T00:00:01.000+0000,1,1307067,2053013558920210000,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,6.0,2.0
2019-10-01T00:00:04.000+0000,1,1004237,2053013555631880000,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,1.0,0.0
2019-10-01T00:00:05.000+0000,1,1480613,2053013561092860000,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9,36.0,9.0
2019-10-01T00:00:10.000+0000,1,28719074,2053013565480100000,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880,33.0,10.0
2019-10-01T00:00:11.000+0000,1,1004545,2053013555631880000,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb,3.0,0.0
2019-10-01T00:00:11.000+0000,1,2900536,2053013554776240000,appliances.kitchen.microwave,elenberg,51.46,555158050,b5bdd0b3-4ca2-4c55-939e-9ce44bb50abd,11.0,30.0
2019-10-01T00:00:11.000+0000,1,1005011,2053013555631880000,electronics.smartphone,samsung,900.64,530282093,50a293fb-5940-41b2-baf3-17af0e812101,0.0,0.0
2019-10-01T00:00:13.000+0000,1,3900746,2053013552326770000,appliances.environment.water_heater,haier,102.38,555444559,98b88fa0-d8fa-4b9d-8a71-3dd403afab85,30.0,26.0
2019-10-01T00:00:16.000+0000,1,13500240,2053013557099880000,furniture.bedroom.bed,brw,93.18,555446365,7f0062d8-ead0-4e0a-96f6-43a0b79a2fc4,38.0,25.0


In [0]:
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- brandIndex: double (nullable = false)
 |-- category_codeIndex: double (nullable = false)



In [0]:
# Create a view or table
temp_table_name = "2019_Oct_1_txt"
df.createOrReplaceTempView(temp_table_name)

In [0]:
if PYSPARK_CLI:
    csv = spark.read.csv('2019_Oct_1_txt', inferSchema=True, header=True)
else:
    csv = spark.sql("SELECT * FROM 2019_Oct_1_txt")

In [0]:
data = csv.select("product_id", "brandIndex", "category_codeIndex", "price", "user_id", col("event_type").alias("label"))

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [0]:
# Split the data
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
print ("Training Rows:", train.count(), " Testing Rows:", test.count())

Training Rows: 459408  Testing Rows: 197925


## DECISION TREE CLASSIFIER

### Prepare the Training Data
To train the regression model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector.

In [0]:
assembler_dt = VectorAssembler(inputCols=["product_id", "brandIndex", "category_codeIndex", "price", "user_id" ], outputCol="features")

#lr = LinearRegression(labelCol="label", featuresCol="normFeatures")
dt = DecisionTreeClassifier(labelCol="label", featuresCol= "features", maxBins=3000)

### Parameter Building and Train using Cross Validator

In [0]:
# TODO: params refered to the reference above
paramGrid_cv_dt = ParamGridBuilder() \
  .addGrid(dt.maxDepth, [3, 5]) \
  .addGrid(dt.minInfoGain, [0.0]) \
  .build()

In [0]:
# Number of folds= 3 for Cross Validator
pipeline_cv_dt = Pipeline(stages=[assembler_dt, dt])

K = 3
cv_dt = CrossValidator(estimator=pipeline_cv_dt, evaluator=MulticlassClassificationEvaluator(), estimatorParamMaps=paramGrid_cv_dt, numFolds = K)

model_cv_dt = cv_dt.fit(train)

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict Event Type where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted event type to the actual event type.

In [0]:
prediction_cv_dt = model_cv_dt.transform(test)
prediction_cv_dt.select("features", "prediction", "trueLabel")

Out[15]: DataFrame[features: vector, prediction: double, trueLabel: int]

### Obtaning Accuracy and Test Error

In [0]:
evaluator_cv_dt = MulticlassClassificationEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy_cv_dt = evaluator_cv_dt.evaluate(prediction_cv_dt)
print ("Average Accuracy =", accuracy_cv_dt)
print ("Test Error = ", (1 - accuracy_cv_dt))

Average Accuracy = 0.9594543387646836
Test Error =  0.040545661235316444


###References

https://towardsdatascience.com/multi-class-text-classification-with-pyspark-7d78d022ed35

https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier